In [61]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statsmodels.api as sm
from scipy.special import gamma, loggamma, factorial
import scipy.stats
from scipy.interpolate import interp1d
from matplotlib.ticker import AutoMinorLocator 
from matplotlib import rc, font_manager
from matplotlib.lines import Line2D
from matplotlib import colors as mcolors
from mycolorpy import colorlist as mcp
from matplotlib import legend_handler
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
import collections
import matplotlib
import shapely.geometry
import shapely.ops 
from geopy.geocoders import Nominatim
import geopandas as gpd
import mapclassify
import jenkspy
import contextily as cx
import os
import rioxarray as rx
import pyreadr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
import plots
import seaborn as sns
from prophet import Prophet
import datetime
import warnings
warnings.filterwarnings("ignore")

In [62]:
wd = os.path.dirname(os.getcwd())

In [122]:
country = "MEX"

if country == "ARG":
    country_full = "argentina"
elif country == "CHL":
    country_full = "chile"
elif country == "COL":
    country_full = "colombia"
elif country == "MEX":
    country_full = "mexico"

In [123]:
date_list = []
years = ['2020', '2021', '2022']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
for year in years:
    i = 0
    if year == '2020':
        i = 3
    if year == '2022':
        max_month = 3
    else:
        max_month = 12
    while i < max_month:
        date_list.append(year + '-' + str(months[i]) + '-01')
        i += 1


In [114]:
df_stringency = pd.read_csv("/Volumes/RECAST/data/covid_stringency/owid-covid-data.csv")
df_stringency = df_stringency[df_stringency['location'] == str(country_full).capitalize()].reset_index(drop=True)
df_stringency = df_stringency.filter(['location', 'date', 'stringency_index'], axis=1)
df_stringency = df_stringency.fillna(0)
df_stringency['year'] = [int(df_stringency['date'][i][0:4]) for i in range(len(df_stringency))]
df_stringency['month'] = [int(df_stringency['date'][i][5:7]) for i in range(len(df_stringency))]

stringencies = []
dss = []
start_date = '2020-03-01'
end_date = '2023-02-01'
date_range = pd.date_range(start=start_date, end=end_date, freq='MS')
df_stringency_month = pd.DataFrame({'ds': date_range})
for i in range(len(df_stringency_month)):
    year = df_stringency_month.loc[i, 'ds'].year
    month = df_stringency_month.loc[i, 'ds'].month
    df_stringency_filter = df_stringency[df_stringency['year'] == year].reset_index(drop=True)
    df_stringency_filter = df_stringency_filter[df_stringency_filter['month'] == month].reset_index(drop=True)
    stringencies.append(np.mean(df_stringency_filter['stringency_index']))
    

In [124]:
gdf_admin_mov = gpd.read_file(wd + '/data/outputs/' + country + '/gdf_' + country + '_admin_rdi_mov_evo.gpkg')
gdf_admin_mov = gdf_admin_mov.sort_values(by = ['rdi'], ascending=True).reset_index(drop=True)

months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
years = ['2020', '2021', '2022']

for year in years:
    for month in months:  
        try:
            gdf_admin_mov = gdf_admin_mov[gdf_admin_mov['z_score_perchange_' + str(month) + '_' + str(year)] < 4].reset_index(drop=True)
        except:
            continue

df_rdi = pd.DataFrame({'rdi': gdf_admin_mov['rdi']})
    
df_evo_admins = {}
for i in range(len(gdf_admin_mov)):
    evo_admin = []
    for year in years:
        for month in months:
            try:
                evo_admin.append(gdf_admin_mov.loc[i, 'n_perchange_' + month + '_' + year]) 
            except:
                continue
    df_evo_admins[str(i)] = pd.DataFrame({'ds': date_list, 'y': evo_admin})
    
    months_evo = []
    for j in range(len(df_evo_admins[str(i)])):
        months_evo.append(datetime.datetime.strptime(df_evo_admins[str(i)]['ds'][j], '%Y-%m-%d').month)
    df_evo_admins[str(i)]['month'] = months_evo
    df_evo_admins[str(i)]['stringency'] = stringencies[0:len(df_evo_admins[str(i)])]
    

In [125]:
df_rdi.to_csv(wd + '/data/outputs/' + country + '/prediction/rdi.csv', index=False)


In [99]:
fcst_admins = {}
for i in range(len(df_evo_admins)):
    m = Prophet(seasonality_mode='additive', yearly_seasonality=True)
#     m.add_regressor('month', standardize=False)
#     m.add_regressor('stringency', standardize=True)
    m.fit(df_evo_admins[str(i)])
    future = m.make_future_dataframe(periods=12, freq='MS')
#     future['month'] = future['ds'].dt.month
#     future['stringency'] = stringencies
    fcst = m.predict(future)
    fcst_admins[str(i)] = fcst

15:36:04 - cmdstanpy - INFO - Chain [1] start processing
15:36:05 - cmdstanpy - INFO - Chain [1] done processing
15:36:05 - cmdstanpy - INFO - Chain [1] start processing
15:36:05 - cmdstanpy - INFO - Chain [1] done processing
15:36:05 - cmdstanpy - INFO - Chain [1] start processing
15:36:05 - cmdstanpy - INFO - Chain [1] done processing
15:36:05 - cmdstanpy - INFO - Chain [1] start processing
15:36:05 - cmdstanpy - INFO - Chain [1] done processing
15:36:05 - cmdstanpy - INFO - Chain [1] start processing
15:36:05 - cmdstanpy - INFO - Chain [1] done processing
15:36:05 - cmdstanpy - INFO - Chain [1] start processing
15:36:05 - cmdstanpy - INFO - Chain [1] done processing
15:36:05 - cmdstanpy - INFO - Chain [1] start processing
15:36:05 - cmdstanpy - INFO - Chain [1] done processing
15:36:05 - cmdstanpy - INFO - Chain [1] start processing
15:36:05 - cmdstanpy - INFO - Chain [1] done processing
15:36:05 - cmdstanpy - INFO - Chain [1] start processing
15:36:06 - cmdstanpy - INFO - Chain [1]

15:36:20 - cmdstanpy - INFO - Chain [1] done processing
15:36:20 - cmdstanpy - INFO - Chain [1] start processing
15:36:20 - cmdstanpy - INFO - Chain [1] done processing
15:36:20 - cmdstanpy - INFO - Chain [1] start processing
15:36:20 - cmdstanpy - INFO - Chain [1] done processing
15:36:20 - cmdstanpy - INFO - Chain [1] start processing
15:36:21 - cmdstanpy - INFO - Chain [1] done processing
15:36:21 - cmdstanpy - INFO - Chain [1] start processing
15:36:21 - cmdstanpy - INFO - Chain [1] done processing
15:36:21 - cmdstanpy - INFO - Chain [1] start processing
15:36:21 - cmdstanpy - INFO - Chain [1] done processing
15:36:21 - cmdstanpy - INFO - Chain [1] start processing
15:36:21 - cmdstanpy - INFO - Chain [1] done processing
15:36:21 - cmdstanpy - INFO - Chain [1] start processing
15:36:21 - cmdstanpy - INFO - Chain [1] done processing
15:36:21 - cmdstanpy - INFO - Chain [1] start processing
15:36:21 - cmdstanpy - INFO - Chain [1] done processing
15:36:21 - cmdstanpy - INFO - Chain [1] 

15:37:03 - cmdstanpy - INFO - Chain [1] done processing
15:37:03 - cmdstanpy - INFO - Chain [1] start processing
15:37:03 - cmdstanpy - INFO - Chain [1] done processing
15:37:03 - cmdstanpy - INFO - Chain [1] start processing
15:37:03 - cmdstanpy - INFO - Chain [1] done processing
15:37:03 - cmdstanpy - INFO - Chain [1] start processing
15:37:10 - cmdstanpy - INFO - Chain [1] done processing
15:37:10 - cmdstanpy - INFO - Chain [1] start processing
15:37:10 - cmdstanpy - INFO - Chain [1] done processing
15:37:10 - cmdstanpy - INFO - Chain [1] start processing
15:37:10 - cmdstanpy - INFO - Chain [1] done processing
15:37:10 - cmdstanpy - INFO - Chain [1] start processing
15:37:10 - cmdstanpy - INFO - Chain [1] done processing
15:37:10 - cmdstanpy - INFO - Chain [1] start processing
15:37:10 - cmdstanpy - INFO - Chain [1] done processing
15:37:10 - cmdstanpy - INFO - Chain [1] start processing
15:37:17 - cmdstanpy - INFO - Chain [1] done processing
15:37:17 - cmdstanpy - INFO - Chain [1] 

15:37:39 - cmdstanpy - INFO - Chain [1] done processing
15:37:39 - cmdstanpy - INFO - Chain [1] start processing
15:37:39 - cmdstanpy - INFO - Chain [1] done processing
15:37:39 - cmdstanpy - INFO - Chain [1] start processing
15:37:39 - cmdstanpy - INFO - Chain [1] done processing
15:37:39 - cmdstanpy - INFO - Chain [1] start processing
15:37:40 - cmdstanpy - INFO - Chain [1] done processing
15:37:40 - cmdstanpy - INFO - Chain [1] start processing
15:37:40 - cmdstanpy - INFO - Chain [1] done processing
15:37:40 - cmdstanpy - INFO - Chain [1] start processing
15:37:40 - cmdstanpy - INFO - Chain [1] done processing
15:37:40 - cmdstanpy - INFO - Chain [1] start processing
15:37:40 - cmdstanpy - INFO - Chain [1] done processing
15:37:40 - cmdstanpy - INFO - Chain [1] start processing
15:37:40 - cmdstanpy - INFO - Chain [1] done processing
15:37:40 - cmdstanpy - INFO - Chain [1] start processing
15:37:40 - cmdstanpy - INFO - Chain [1] done processing
15:37:40 - cmdstanpy - INFO - Chain [1] 

15:38:08 - cmdstanpy - INFO - Chain [1] done processing
15:38:08 - cmdstanpy - INFO - Chain [1] start processing
15:38:09 - cmdstanpy - INFO - Chain [1] done processing
15:38:09 - cmdstanpy - INFO - Chain [1] start processing
15:38:09 - cmdstanpy - INFO - Chain [1] done processing
15:38:09 - cmdstanpy - INFO - Chain [1] start processing
15:38:16 - cmdstanpy - INFO - Chain [1] done processing
15:38:16 - cmdstanpy - INFO - Chain [1] start processing
15:38:16 - cmdstanpy - INFO - Chain [1] done processing
15:38:16 - cmdstanpy - INFO - Chain [1] start processing
15:38:16 - cmdstanpy - INFO - Chain [1] done processing
15:38:16 - cmdstanpy - INFO - Chain [1] start processing
15:38:16 - cmdstanpy - INFO - Chain [1] done processing
15:38:16 - cmdstanpy - INFO - Chain [1] start processing
15:38:16 - cmdstanpy - INFO - Chain [1] done processing
15:38:16 - cmdstanpy - INFO - Chain [1] start processing
15:38:16 - cmdstanpy - INFO - Chain [1] done processing
15:38:16 - cmdstanpy - INFO - Chain [1] 

...

15:39:48 - cmdstanpy - INFO - Chain [1] done processing
15:39:48 - cmdstanpy - INFO - Chain [1] start processing
15:39:55 - cmdstanpy - INFO - Chain [1] done processing
15:39:55 - cmdstanpy - INFO - Chain [1] start processing
15:39:55 - cmdstanpy - INFO - Chain [1] done processing
15:39:55 - cmdstanpy - INFO - Chain [1] start processing
15:39:55 - cmdstanpy - INFO - Chain [1] done processing
15:39:55 - cmdstanpy - INFO - Chain [1] start processing
15:39:55 - cmdstanpy - INFO - Chain [1] done processing
15:39:55 - cmdstanpy - INFO - Chain [1] start processing
15:39:55 - cmdstanpy - INFO - Chain [1] done processing
15:39:55 - cmdstanpy - INFO - Chain [1] start processing
15:39:55 - cmdstanpy - INFO - Chain [1] done processing
15:39:55 - cmdstanpy - INFO - Chain [1] start processing
15:39:55 - cmdstanpy - INFO - Chain [1] done processing
15:39:55 - cmdstanpy - INFO - Chain [1] start processing
15:39:56 - cmdstanpy - INFO - Chain [1] done processing
15:39:56 - cmdstanpy - INFO - Chain [1] 

15:40:45 - cmdstanpy - INFO - Chain [1] done processing
15:40:45 - cmdstanpy - INFO - Chain [1] start processing
15:40:45 - cmdstanpy - INFO - Chain [1] done processing
15:40:45 - cmdstanpy - INFO - Chain [1] start processing
15:40:45 - cmdstanpy - INFO - Chain [1] done processing
15:40:45 - cmdstanpy - INFO - Chain [1] start processing
15:40:45 - cmdstanpy - INFO - Chain [1] done processing
15:40:45 - cmdstanpy - INFO - Chain [1] start processing
15:40:45 - cmdstanpy - INFO - Chain [1] done processing
15:40:45 - cmdstanpy - INFO - Chain [1] start processing
15:40:45 - cmdstanpy - INFO - Chain [1] done processing
15:40:45 - cmdstanpy - INFO - Chain [1] start processing
15:40:45 - cmdstanpy - INFO - Chain [1] done processing
15:40:46 - cmdstanpy - INFO - Chain [1] start processing
15:40:46 - cmdstanpy - INFO - Chain [1] done processing
15:40:46 - cmdstanpy - INFO - Chain [1] start processing
15:40:46 - cmdstanpy - INFO - Chain [1] done processing
15:40:46 - cmdstanpy - INFO - Chain [1] 

15:41:07 - cmdstanpy - INFO - Chain [1] done processing
15:41:07 - cmdstanpy - INFO - Chain [1] start processing
15:41:07 - cmdstanpy - INFO - Chain [1] done processing
15:41:07 - cmdstanpy - INFO - Chain [1] start processing
15:41:07 - cmdstanpy - INFO - Chain [1] done processing
15:41:07 - cmdstanpy - INFO - Chain [1] start processing
15:41:07 - cmdstanpy - INFO - Chain [1] done processing
15:41:07 - cmdstanpy - INFO - Chain [1] start processing
15:41:07 - cmdstanpy - INFO - Chain [1] done processing
15:41:07 - cmdstanpy - INFO - Chain [1] start processing
15:41:07 - cmdstanpy - INFO - Chain [1] done processing
15:41:07 - cmdstanpy - INFO - Chain [1] start processing
15:41:08 - cmdstanpy - INFO - Chain [1] done processing
15:41:08 - cmdstanpy - INFO - Chain [1] start processing
15:41:08 - cmdstanpy - INFO - Chain [1] done processing
15:41:08 - cmdstanpy - INFO - Chain [1] start processing
15:41:08 - cmdstanpy - INFO - Chain [1] done processing
15:41:08 - cmdstanpy - INFO - Chain [1] 

15:42:08 - cmdstanpy - INFO - Chain [1] done processing
15:42:08 - cmdstanpy - INFO - Chain [1] start processing
15:42:08 - cmdstanpy - INFO - Chain [1] done processing
15:42:08 - cmdstanpy - INFO - Chain [1] start processing
15:42:08 - cmdstanpy - INFO - Chain [1] done processing
15:42:08 - cmdstanpy - INFO - Chain [1] start processing
15:42:09 - cmdstanpy - INFO - Chain [1] done processing
15:42:09 - cmdstanpy - INFO - Chain [1] start processing
15:42:09 - cmdstanpy - INFO - Chain [1] done processing
15:42:09 - cmdstanpy - INFO - Chain [1] start processing
15:42:09 - cmdstanpy - INFO - Chain [1] done processing
15:42:09 - cmdstanpy - INFO - Chain [1] start processing
15:42:09 - cmdstanpy - INFO - Chain [1] done processing
15:42:09 - cmdstanpy - INFO - Chain [1] start processing
15:42:09 - cmdstanpy - INFO - Chain [1] done processing
15:42:09 - cmdstanpy - INFO - Chain [1] start processing
15:42:09 - cmdstanpy - INFO - Chain [1] done processing
15:42:09 - cmdstanpy - INFO - Chain [1] 

15:43:04 - cmdstanpy - INFO - Chain [1] done processing
15:43:04 - cmdstanpy - INFO - Chain [1] start processing
15:43:04 - cmdstanpy - INFO - Chain [1] done processing
15:43:04 - cmdstanpy - INFO - Chain [1] start processing
15:43:04 - cmdstanpy - INFO - Chain [1] done processing
15:43:04 - cmdstanpy - INFO - Chain [1] start processing
15:43:04 - cmdstanpy - INFO - Chain [1] done processing
15:43:04 - cmdstanpy - INFO - Chain [1] start processing
15:43:04 - cmdstanpy - INFO - Chain [1] done processing


In [100]:
for i in range(len(df_evo_admins)):
    fcst_admins[str(i)].to_csv(wd + '/data/outputs/' + country + '/prediction/admin_' + str(i) + '.csv', index=False)
